# Genbase
Esse Jupyter Notebook apresenta nosso estudo e os resultados obtidos para o conjunto de dados *Genbase*.

In [ ]:
# Instalação dos pacotes
#!pip install scikit-mutilearn
#!pip install scipy
#!pip install pandas
#!pip install matplotlib
#!pip install numpy
#!pip install -U scikit-learn

## Leitura dos dados e Pré-processamento

A função convert é responsável por transformar dados categóricos e binários para dados numéricos.

In [96]:
def convert(a):
    # tenta converter para uma string
    try:
        b = a.decode("utf-8")
    except:
        # eh um numero
        return a
    # tenta converter para um inteiro
    try:
        return int(b)
    except:
        # eh um atributo nominal
        if b == 'YES':
            return 1
        if b == 'NO':
            return 0
        return b

In [97]:
# Imports necessários para extraçao dos dados
import scipy
from scipy.io import arff
import pandas as pd

# Carregando o treino
data_train, meta_train = scipy.io.arff.loadarff(f'datasets/genbase/genbase-train.arff')
X_train = pd.DataFrame(data_train)

# Carregando o teste
data_test, meta_test = scipy.io.arff.loadarff(f'datasets/genbase/genbase-test.arff')
X_test = pd.DataFrame(data_test)

# Pré-processamento
# Transformando em atributos numéricos
X_test = X_test.applymap(convert)
X_train = X_train.applymap(convert)

# Vamos esquecer por enquanto a coluna "protein", 
# pois serve apenas para identificação da instância
# e pode atrapalhar nos cálculos dos algoritmos.
protein_test = X_test.pop('protein')
protein_train = X_train.pop('protein')

# # Separando o Y do treino
Y_train = X_train.iloc[:, -27:]
X_train.drop(columns=list(Y_train.columns), inplace=True)

# # Separando o Y do teste
Y_test = X_test.iloc[:, -27:]
X_test.drop(columns=list(Y_test.columns), inplace=True)


In [98]:
# Verificamos os rótulos do teste
Y_test

,PDOC00154,PDOC00343,PDOC00271,PDOC00064,PDOC00791,PDOC00380,PDOC50007,PDOC00224,PDOC00100,PDOC00670,...,PDOC00662,PDOC00018,PDOC50001,PDOC00014,PDOC00750,PDOC50196,PDOC50199,PDOC00660,PDOC00653,PDOC00030
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Algoritmos
Vamos estudar esse conjunto de dados com três métodos de classificação multirrótulo:
- Binary Relevance
- Classifier Chains
- Label Powersets.

Também utilizamos três classificadores de um rótulo para cada classificador multirrótulo, são eles:
- Multinomial Naive Bayes
- Support Vector Classification
- Random Forest Classifier. 

In [104]:
# Começamos com o Binary Relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Realizamos uma Cross Validation para determinarmos os melhores
# parâmetros de execução do algoritmo de classificação multirrótulo.
# Dentre esses parâmetros, está o classificador que vamos utilizar
# para resolver o problema após conversão para um rótulo.


grid_param = [ 
    {
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.7, 1.0],
    },
    {
        'classifier': [SVC()],
        'classifier__kernel': ['rbf', 'linear'],
        'classifier__C': [1, 10],
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__bootstrap': [True, False],
    }
]

br = GridSearchCV(BinaryRelevance(), grid_param, scoring='accuracy')
br.fit(X_train, Y_train)



C:\Users\Mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MultinomialNB was fitted without feature names
  warnings.warn(
C:\Users\Mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MultinomialNB was fitted without feature names
  warnings.warn(
C:\Users\Mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MultinomialNB was fitted without feature names
  warnings.warn(
C:\Users\Mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MultinomialNB wa

GridSearchCV(estimator=BinaryRelevance(require_dense=[True, True]),
             param_grid=[{'classifier': [MultinomialNB()],
                          'classifier__alpha': [0.7, 1.0]},
                         {'classifier': [SVC()], 'classifier__C': [1, 10],
                          'classifier__kernel': ['rbf', 'linear']},
                         {'classifier': [RandomForestClassifier(criterion='entropy')],
                          'classifier__bootstrap': [True, False],
                          'classifier__criterion': ['gini', 'entropy']}],
             scoring='accuracy')

In [100]:
# Valor de acurácia da melhor escolha do Cross Validation
br.best_score_

0.9784244974287051

In [101]:
# Melhores parâmetros escolhidos pelo Cross Validation
br.best_params_

{'classifier': RandomForestClassifier(),
 'classifier__bootstrap': True,
 'classifier__criterion': 'gini'}

In [102]:
# Melhores parâmetros escolhidos pelo Cross Validation
# De forma simplificada
br.best_estimator_

BinaryRelevance(classifier=RandomForestClassifier(), require_dense=[True, True])

In [105]:
# Realiza o fit com os parâmetros escolhidos 
br = BinaryRelevance(classifier=br.best_params_['classifier'](criterion=
br.best_params_[''], require_dense=[True, True]))
br.fit(X_train, Y_train)


TypeError: BinaryRelevance.__init__() takes from 1 to 3 positional arguments but 4 were given

In [ ]:
# Testa a qualidade da solução
from sklearn.metrics import accuracy_score

# Verificar acurácia
pred = br.predict(X_test)
accuracy_score(Y_test, pred)


In [ ]:
# Verificar o Hamming Loss
from sklearn.metrics import hamming_loss
hamming_loss(Y_test, pred)

In [ ]:
# Vamos utilizar o Classifier Chains
from skmultilearn.problem_transform import ClassifierChain
# Realizamos uma Cross Validation para determinarmos os melhores
# parâmetros de execução do algoritmo de classificação multirrótulo.
# Dentre esses parâmetros, está o classificador que vamos utilizar
# para resolver o problema após conversão para um rótulo.

cc = GridSearchCV(ClassifierChain(), grid_param, scoring='accuracy')
cc.fit(X_train, Y_train)

In [ ]:
cc.best_score_

In [ ]:
cc.best_params_

In [ ]:
cc.best_estimator_

In [ ]:
# Realiza o fit com os parâmetros escolhidos 
cc = ClassifierChain(classifier=RandomForestClassifier(criterion='entropy'), require_dense=[True, True])
cc.fit(X_train, Y_train)

In [ ]:
# Verificar acurácia
pred2 = cc.predict(X_test)
accuracy_score(Y_test, pred2)

In [ ]:
# Verificar o Hamming Loss
hamming_loss(Y_test, pred)

In [ ]:
# Vamos utilizar o Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# Realizamos uma Cross Validation para determinarmos os melhores
# parâmetros de execução do algoritmo de classificação multirrótulo.
# Dentre esses parâmetros, está o classificador que vamos utilizar
# para resolver o problema após conversão para um rótulo.


lp = GridSearchCV(LabelPowerset(), grid_param, scoring='accuracy')
lp.fit(X_train, Y_train)

In [ ]:
lp.best_score_

In [ ]:
lp.best_params_

In [ ]:
lp.best_estimator_

In [ ]:
# Realiza o fit com os parâmetros escolhidos 
lp = LabelPowerset(classifier=SVC(C=10), require_dense=[True, True])
lp.fit(X_train, Y_train)

In [ ]:
# Verificar acurácia
pred3 = lp.predict(X_test)
accuracy_score(Y_test, pred3)

In [ ]:
# Verificar o Hamming Loss
hamming_loss(Y_test, pred3)